# Phone

In [1]:
import util

1000


The util.py file contains a couple useful methods and packages. It contains the entire socket package, the entire Cryptodome package, as well as the following methods: 
- encrypt(message, key, nonce)
- decrypt(ciphertext, key, nonce)
- hash(message, key, nonce)
- generate_n()
- generate_k()
These methods all use various Cryptodome features, with several constants to ensure we're building based on the same model design.

In [11]:
k_ma = util.generate_k()
n = util.generate_n()
req = b"thisisamessage"

h = util.hash(n, req, k_ma)
m = util.encrypt(req, k_ma, n)

client_socket = util.socket.socket(util.socket.AF_INET, util.socket.SOCK_STREAM)
client_socket.connect((util.SERVER["alias"], util.SERVER["port"]))

print(h)
print(m)

client_socket.sendall(n + m + h)

client_socket.close()

b'B\x85F \xaa\xd7@ \x81\x1e~\x0cZC\x95\xfc)\xf3\xaa{\x08\x8c\x10\xb9\x8e\xe8Oa\xb1\xa6qh'
b'\xc8U\x9c\xe5:\x19\xae\xe4\t\x02U\xd5\xf9\xb4'


CODE IN DEVELOPMENT

In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA256
import socket

from util import SERVER, generate_n

import util

def phone():
    private_key = util.PHONE_PRIVATE_KEY
    public_key = util.PUBLIC_KEY

    print("phone's RSA keys:")
    #print(f"Private Key: {private_key.decode()}")
    #print(f"Public Key: {public_key.decode()}")

    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((SERVER["alias"], SERVER["port"]))

    nonce_p = generate_n()
    cipher = PKCS1_OAEP.new(RSA.import_key(public_key))
    encrypted_nonce_p = cipher.encrypt(nonce_p)
    sock.send(encrypted_nonce_p + public_key)

    received_data = sock.recv(1024)
    encrypted_nonce_s = received_data[:256]
    server_public_key = received_data[256:]

    print(received_data)
    print("\n")
    print((encrypted_nonce_s))
    print("\n")
    print(server_public_key)

    cipher = PKCS1_OAEP.new(RSA.import_key(private_key))
    nonce_s = cipher.decrypt(encrypted_nonce_s)

    cipher = PKCS1_OAEP.new(RSA.import_key(server_public_key))
    encrypted_nonce_s = cipher.encrypt(nonce_s)
    sock.send(encrypted_nonce_s)

    print("3-way handshake completed")

    message = b"bruh"
    cipher = PKCS1_OAEP.new(RSA.import_key(server_public_key))
    encrypted_message = cipher.encrypt(message)
    sock.send(encrypted_message)

    print("message sent to server")

if __name__ == "__main__":
    phone()

phone's RSA keys:
